## Nighttime Lights Trends in Gaza

Analyzing conflict dynamics through the lens of [NASA's Black Marble Nighttime Lights](https://blackmarble.gsfc.nasa.gov) dataset opens a unique window into the often-hidden facets of global unrest. In a world marked by diverse forms of conflict, from armed confrontations to civil unrest, the dataset offers an unconventional yet powerful tool for understanding the ripple effects of these conflicts on human settlements and infrastructure. By tracking nighttime light variations and disruptions, we can unearth vital insights into population displacement, economic destabilization, and the societal impacts of conflict. This analysis explores the potential of the Black Marble Nighttime Lights dataset to not only detect areas affected by conflict but also to quantify the extent of its influence on human lives and livelihoods, providing a valuable perspective on the multifaceted consequences of conflict worldwide.

In [1]:
import math
import os
from datetime import datetime
from math import pi

import colorcet as cc
import dask.dataframe as dd
import folium
import geopandas
import numpy as np
import pandas as pd
from blackmarble.bm_extract import bm_extract
from blackmarble.bm_raster import bm_raster
from bokeh.layouts import column as b_column
from bokeh.layouts import gridplot
from bokeh.models import (
    BasicTicker,
    ColumnDataSource,
    Div,
    HoverTool,
    Legend,
    PrintfTickFormatter,
    Range1d,
    Span,
    TabPanel,
    Tabs,
    Text,
    Title,
)
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import linear_cmap

In [2]:
group = lambda flat, size: [flat[i : i + size] for i in range(0, len(flat), size)]

## Data

### Define Region of Interest

Define region of interest for where NASA Black Marble will be downloaded.

In [3]:
PSE = geopandas.read_file("../../data/boundaries/gadm41_PSE_shp/gadm41_PSE_2.shp")
PSE.explore()


### Black Marble 

[NASA's Black Marble](https://blackmarble.gsfc.nasa.gov) VIIRS (Visible Infrared Imaging Radiometer Suite) Nighttime Lights dataset represents a remarkable advancement in our ability to monitor and understand nocturnal light emissions on a global scale. By utilizing cutting-edge satellite technology and image processing techniques, the Black Marble VIIRS dataset offers a comprehensive and high-resolution view of the Earth's nighttime illumination patterns. 

In [4]:
dates = (
    pd.date_range("2022-01-01", "2023-11-14", freq="D").strftime("%Y-%m-%d").tolist()
)

df = bm_extract(
    roi_sf=ROI,
    product_id="VNP46A2",
    date=dates,
    bearer=os.environ.get("BLACKMARBLE_TOKEN"),
    output_location_type="file",
    file_dir="data",
    file_prefix="pse2_",
    aggregation_fun=["count", "mean", "min", "max", "median", "sum"],
    quiet=True,
)

In [5]:
df = dd.read_csv(
    "data/pse2_VNP46A2*.csv",
    parse_dates=["date"],
).compute()
df = df[df["NAME_1"] == "Gaza"]

## Methodology

Creating a time series of weekly radiance using NASA's Black Marble data involves several steps, including data acquisition, pre-processing, zonal statistics calculation, and time series generation. Below is a general methodology for this process.

###  Time Series Generation

Organize the zonal statistics results in a tabular format, where each columnn corresponds to a specific zone, and rows represent the daily radiance values. Next, we aggregate the data on a weekly basis, computing the desired statistical metric (e.g., mean radiance) for each zone for each week. Finally, we will visualize the time series data to observe trends, patterns, and anomalies over time.

In [6]:
NTL = (
    df.pivot_table(values=["ntl_mean"], index="date", columns=["NAME_2"])
    .resample("W", label="left")
    .mean()
)
NTL

ntl_mean                                              \
NAME_2     Deir Al-Balah        Gaza Gaza ash Shamaliyah  Khan Yunis   
date                                                                   
2021-12-26     80.876536  135.911132           78.008197  121.089891   
2022-01-02    133.175823  203.156980          118.514480  135.848382   
2022-01-09    120.802306  154.040451          108.742094  129.915630   
2022-01-16    134.793033  197.818275          131.700803  123.835625   
2022-01-23     97.599245  151.641740           94.552836  108.125086   
...                  ...         ...                 ...         ...   
2023-10-01     97.974453  189.140499          145.523352  104.302415   
2023-10-08     18.601694   33.387448           20.320082   26.547027   
2023-10-15     15.493020   31.566008           22.924328   18.401297   
2023-10-22     19.406778   61.449688           30.709801   27.215036   
2023-10-29     21.139765   71.703914           39.983443   29.838535   

                        
NAME_2           Rafah  
date                    
2021-12-26         NaN  
2022-01-02  225.478376  
2022-01-09  218.354316  
2022-01-16  201.245460  
2022-01-23  165.917218  
...                ...  
2023-10-01  189.881470  
2023-10-08   52.033815  
2023-10-15   48.671099  
2023-10-22   58.474931  
2023-10-29   61.242155  

[97 rows x 5 columns]

## Findings



### Percent Change in NTL Radiance Average 

n this exploratory analysis, we conducted analysis of NTL radiance trends, comparing the observed radiance levels to a baseline established in the year 2022.

In [7]:
data = 100 * (
    NTL / NTL[(NTL.index >= "2022-01-01") & (NTL.index < "2023-01-01")].mean() - 1
)
data

ntl_mean                                                     
NAME_2     Deir Al-Balah       Gaza Gaza ash Shamaliyah Khan Yunis      Rafah
date                                                                         
2021-12-26    -37.204006 -38.365424          -46.229746  -6.184716        NaN
2022-01-02      3.403392  -7.869988          -18.309177   5.249534  11.442919
2022-01-09     -6.203935 -30.143928          -25.045183   0.653091   7.921845
2022-01-16      4.659063 -10.291046           -9.219980  -4.057438  -0.534224
2022-01-23    -24.219781 -31.231724          -34.825694 -16.229294 -17.995245
...                  ...        ...                 ...        ...        ...
2023-10-01    -23.928454 -14.226346            0.307762 -19.190937  -6.150888
2023-10-08    -85.556851 -84.859068          -85.993575 -79.432495 -74.282234
2023-10-15    -87.970558 -85.685076          -84.198495 -85.743460 -75.944259
2023-10-22    -84.931750 -72.133074          -78.832047 -78.914951 -71.098705
2023-10-29    -83.586185 -67.482867          -72.439821 -76.882375 -69.731002

[97 rows x 5 columns]

In [8]:
def create_plot(data):
    p = figure(
        title="Palestine: Percent Change in Nighttime Lights Radiance Average",
        width=800,
        height=800,
        x_axis_label="Date",
        x_axis_type="datetime",
        y_axis_label="NTL Radiance Percent change (%)",
        tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
    )
    p.y_range = Range1d(-100, 50, bounds=(-100, 100))
    p.xaxis.major_label_orientation = math.pi / 4
    p.add_layout(
        Title(
            text=f"Percent change in NTL radiance for each second-level administrative division",
            text_font_size="12pt",
            text_font_style="italic",
        ),
        "above",
    )
    p.add_layout(
        Title(
            text=f"Source: NASA Black Marble. Creation date: {datetime.today().strftime('%d %B %Y')}. Feedback: datalab@worldbank.org.",
            text_font_size="10pt",
            text_font_style="italic",
        ),
        "below",
    )
    p.add_layout(Legend(), "right")
    p.renderers.extend(
        [
            Span(
                location=datetime(2023, 10, 7),
                dimension="height",
                line_color="gray",
                line_width=1.5,
                line_dash=(4, 4),
            ),
        ]
    )
    p.add_tools(
        HoverTool(
            tooltips="Date: @x{%F}, Percent Change: @y",
            formatters={"@x": "datetime"},
        )
    )
    renderers = []
    for column, color in zip(data.columns, cc.b_glasbey_category10):
        r = p.line(
            data.index,
            data[column],
            legend_label=str(column[1]),
            line_color=color,
            line_width=2,
        )
        renderers.append(r)

    p.legend.location = "bottom_left"
    p.legend.click_policy = "hide"
    p.title.text_font_size = "12pt"
    # p.sizing_mode = "scale_both"
    return p

In [9]:
output_notebook()
show(create_plot(data))

Loading BokehJS ...

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Percent change (compared to 2022) in NTL radiance (NASA Black Marble VNP46A2) for each second-level administrative division. 
```

In [10]:
def create_plot(data, colors):
    p = figure(
        title=data.columns[0][1],
        width=800,
        height=600,
        x_axis_label="Date",
        x_axis_type="datetime",
        y_axis_label="NTL Radiance Percent Change (%)",
        tools="pan,wheel_zoom,box_zoom,reset,save,box_select",
    )
    p.y_range = Range1d(-100, 50, bounds=(-100, 100))
    p.xaxis.major_label_orientation = math.pi / 4
    p.add_layout(Legend(), "right")
    p.renderers.extend(
        [
            Span(
                location=datetime(2023, 10, 7),
                dimension="height",
                line_color="red",
                line_width=1.5,
                line_dash=(4, 4),
            ),
        ]
    )
    p.add_tools(
        HoverTool(
            tooltips="date: @x{%F}, percent change: @y",
            formatters={"@x": "datetime"},
        )
    )
    renderers = []
    for column, color in zip(data.columns, colors):
        r = p.line(
            data.index,
            data[column],
            line_color=color,
            line_width=2,
        )
        renderers.append(r)

    p.legend.location = "bottom_left"
    p.legend.click_policy = "hide"
    p.title.text_font_size = "12pt"
    p.sizing_mode = "scale_both"
    return p

In [11]:
plots = list()

for column, color in zip(data.columns, cc.b_glasbey_category10):
    p = create_plot(data[column].to_frame(), [color])
    plots.append(p)

p = gridplot(group(plots, 2))
p.sizing_mode = "scale_both"

In [12]:
show(p)

```{figure} ../../docs/images/logo.png
---
height: 0px
---
Percent change (compared to 2022) in NTL radiance (NASA Black Marble VNP46A2) for each second-level administrative division. 
```

#### Heatmap

In [13]:
df = data.stack().reset_index()
df["date"] = df["date"].astype(str)
df["NAME_2"] = df["NAME_2"].astype(str)

In [14]:
from palettable.colorbrewer.diverging import RdBu_7 as palette

colors = palette.hex_colors

p = figure(
    title=f"Palestine: Weekly Percent Change (compared to 2022 average) in Nighttime Lights Radiance",
    x_range=df["date"].unique(),
    y_range=df["NAME_2"].unique(),
    x_axis_location="above",
    width=1800,
    height=800,
    tools="hover,save,pan,box_zoom,reset,wheel_zoom",
    toolbar_location="below",
    tooltips=[("date", "@date @NAME_2"), ("rate", "@ntl_mean%")],
)

p.title.text_font_size = "18pt"
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "16px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

r = p.rect(
    x="date",
    y="NAME_2",
    width=1,
    height=1,
    source=df,
    fill_color=linear_cmap("ntl_mean", colors, low=-75, high=75),
    line_color=None,
)

p.add_layout(
    Title(
        text=f"Source: NASA Black Marble. Creation date: {datetime.today().strftime('%d %B %Y')}. Feedback: datalab@worldbank.org.",
        text_font_size="10pt",
        text_font_style="italic",
    ),
    "below",
)
p.add_layout(
    r.construct_color_bar(
        major_label_text_font_size="16px",
        ticker=BasicTicker(desired_num_ticks=len(colors)),
        formatter=PrintfTickFormatter(format="%d%%"),
        label_standoff=6,
        border_line_color=None,
        padding=50,
    ),
    "right",
)

In [15]:
show(p)

### Point-in-Time Comparison

#### Daily

```{figure} ./figures/pse_ntl_VNP46A1_2023-01-01.png
---
height: 400px
---
Nighttime lights on January 1, 2023. Source: NASA Black Marble (VNP46A1).
```

```{figure} ./figures/pse_ntl_VNP46A1_2023-10-21.png
---
height: 400px
---
Nighttime lights on October 21, 2023. Source: NASA Black Marble (VNP46A1).
```

#### Weekly
We visualize below weekly snapshots of the percent change (compared to 2022) in NTL radiance average for each second-level administrative division. 

In [16]:
m = PSE.merge(
    data.iloc[1].to_frame("percent change in radiance average (%)").reset_index(),
    on="NAME_2",
).explore(
    column="percent change in radiance average (%)", cmap="viridis", vmin=-100, vmax=0
)

title_html = f"""<h3 align="center" style="font-size:20px">Week of <b>{data.iloc[1].name.strftime("%F")}</b></h3>"""
m.get_root().html.add_child(folium.Element(title_html))
m

In [17]:
m = PSE.merge(
    data.loc["2023-10-08"].to_frame("percent change in radiance average (%)").reset_index(),
    on="NAME_2",
).explore(
    column="percent change in radiance average (%)", cmap="viridis", vmin=-100, vmax=0
)

title_html = f"""<h3 align="center" style="font-size:20px">Week of <b>{data.loc["2023-10-08"].name.strftime("%F")}</b></h3>"""
m.get_root().html.add_child(folium.Element(title_html))
m

In [18]:
m = PSE.merge(
    data.iloc[-1].to_frame("percent change in radiance average (%)").reset_index(),
    on="NAME_2",
).explore(
    column="percent change in radiance average (%)", cmap="viridis", vmin=-100, vmax=0
)

title_html = f"""<h3 align="center" style="font-size:20px">Week of <b>{data.iloc[-1].name.strftime("%F")}</b></h3>"""
m.get_root().html.add_child(folium.Element(title_html))
m

# Limitations 

Using nighttime lights to estimate macroeconomic indicators during conflict may be a valuable approach, but it comes with several assumptions and limitations. Here's a list of some of the key assumptions and limitations:

```{caution}
**Assumptions:**

- **Luminosity Reflects Economic Activity:** The approach assumes that the level of nighttime lights is a reliable proxy for economic activity. It presupposes that areas with brighter lights correspond to higher economic productivity.

- **Baseline Data Availability:** It assumes the availability of baseline nighttime lights data before the onset of the conflict. The accuracy of the estimates depends on the quality and relevance of this baseline data.

- **Spatial Distribution:** The method assumes that nighttime lights are evenly distributed within a given geographic area and that changes in luminosity accurately reflect changes in economic activity across all locations.
    

**Limitations:**

- **Confounding Factors and Data Interpretation:** The approach may require subjective interpretation, as it may not distinguish between reduced lighting due to conflict and reduced lighting due to other factors. Changes in nighttime lights can be influenced by factors other than economic activity, such as energy conservation measures, urban development, or seasonal variations.

- **Generalization:** The approach might lead to overgeneralization, as a reduction in nighttime lights can be associated with various economic outcomes, from minor disruptions to severe economic downturns.

- **Alternative Explanations:** Changes in nighttime lights can result from factors other than conflict, such as urban development, changes in economic activities, or natural disasters. Therefore, it may not always be clear whether a decline in nighttime lights is solely due to conflict.

- **Geopolitical Factors:** The dataset may be subject to geopolitical biases, with some areas having less comprehensive coverage due to political reasons.

- **Data Lag:** There can be a significant time lag between the occurrence of a conflict event and its reflection in the nighttime lights dataset. This lag may limit the dataset's utility for real-time conflict monitoring.

- **Resolution and Urban Bias:** The dataset's spatial resolution may not be fine enough to capture small villages or isolated conflict events. It may also have an urban bias, making it less suitable for analyzing rural or remote conflicts.
```

To address these assumptions and limitations, it is crucial to complement nighttime lights data analysis with other sources of information and adopt a cautious and context-aware approach when interpreting the findings.

# References

{cite:empty}`ROMAN2018113`

```{bibliography}
:filter: docname in docnames
:style: plain
```